In [ ]:
#Input data

In [22]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/DAY48'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/DAY48\test.csv
/DAY48\test.csv.zip
/DAY48\train.csv
/DAY48\trainLabels.csv


In [23]:
#read data

In [24]:
tr=pd.read_csv("/DAY48/train.csv",header=None)
ts=pd.read_csv("/DAY48/test.csv",header=None)
trlabel=pd.read_csv("/DAY48/trainLabels.csv",header=None,names=["Target"])

In [25]:
tr.shape

(1000, 40)

In [26]:
trlabel.shape

(1000, 1)

In [27]:
tr=pd.concat([tr,trlabel],axis=1) ## Concatenation along columns

In [28]:
tr.info() ## No null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 41 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1000 non-null   float64
 1   1       1000 non-null   float64
 2   2       1000 non-null   float64
 3   3       1000 non-null   float64
 4   4       1000 non-null   float64
 5   5       1000 non-null   float64
 6   6       1000 non-null   float64
 7   7       1000 non-null   float64
 8   8       1000 non-null   float64
 9   9       1000 non-null   float64
 10  10      1000 non-null   float64
 11  11      1000 non-null   float64
 12  12      1000 non-null   float64
 13  13      1000 non-null   float64
 14  14      1000 non-null   float64
 15  15      1000 non-null   float64
 16  16      1000 non-null   float64
 17  17      1000 non-null   float64
 18  18      1000 non-null   float64
 19  19      1000 non-null   float64
 20  20      1000 non-null   float64
 21  21      1000 non-null   float64
 22  2

In [29]:
#Modelling

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, roc_auc_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV

In [31]:
#iloc提取最後一欄之前所有資料
tr.iloc[:,:-1]

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.299403,-1.226624,1.498425,-1.176150,5.289853,0.208297,2.404498,1.594506,-0.051608,0.663234,...,-0.850465,-0.622990,-1.833057,0.293024,3.552681,0.717611,3.305972,-2.715559,-2.682409,0.101050
1,-1.174176,0.332157,0.949919,-1.285328,2.199061,-0.151268,-0.427039,2.619246,-0.765884,-0.093780,...,-0.819750,0.012037,2.038836,0.468579,-0.517657,0.422326,0.803699,1.213219,1.382932,-1.817761
2,1.192222,-0.414371,0.067054,-2.233568,3.658881,0.089007,0.203439,-4.219054,-1.184919,-1.240310,...,-0.604501,0.750054,-3.360521,0.856988,-2.751451,-1.582735,1.672246,0.656438,-0.932473,2.987436
3,1.573270,-0.580318,-0.866332,-0.603812,3.125716,0.870321,-0.161992,4.499666,1.038741,-1.092716,...,1.022959,1.275598,-3.480110,-1.065252,2.153133,1.563539,2.767117,0.215748,0.619645,1.883397
4,-0.613071,-0.644204,1.112558,-0.032397,3.490142,-0.011935,1.443521,-4.290282,-1.761308,0.807652,...,0.513906,-1.803473,0.518579,-0.205029,-4.744566,-1.520015,1.830651,0.870772,-1.894609,0.408332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.310429,0.826811,-0.952245,0.768850,1.877520,1.320646,1.944609,1.191420,-0.127724,0.070937,...,-0.600411,-0.383792,0.745596,-0.698598,-2.729937,-0.431535,0.372873,1.019092,-2.672811,-0.295141
996,-1.853879,0.246726,0.459921,-2.074267,7.599220,-0.138355,-4.501900,0.630634,-1.590533,-1.112949,...,0.361736,0.240052,-0.856196,-0.072481,-2.935896,0.582411,-2.613407,0.036687,2.809310,4.412567
997,0.912748,-1.734039,-1.047035,0.217573,13.457812,0.162771,-2.250521,2.216161,-0.378326,0.642114,...,1.195896,-1.073806,-2.754369,1.814864,-4.190105,-1.116441,-2.100125,0.061513,0.895536,0.813686
998,2.439780,-0.735511,-0.902426,1.365036,-10.430299,-0.856859,2.686474,0.292035,0.585388,-0.876965,...,2.262326,-0.039488,0.773876,-0.916066,2.604827,-0.649874,-3.423674,0.229748,-2.311088,-3.422217


In [32]:
tr.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,Target
0,0.299403,-1.226624,1.498425,-1.176150,5.289853,0.208297,2.404498,1.594506,-0.051608,0.663234,...,-0.622990,-1.833057,0.293024,3.552681,0.717611,3.305972,-2.715559,-2.682409,0.101050,1
1,-1.174176,0.332157,0.949919,-1.285328,2.199061,-0.151268,-0.427039,2.619246,-0.765884,-0.093780,...,0.012037,2.038836,0.468579,-0.517657,0.422326,0.803699,1.213219,1.382932,-1.817761,0
2,1.192222,-0.414371,0.067054,-2.233568,3.658881,0.089007,0.203439,-4.219054,-1.184919,-1.240310,...,0.750054,-3.360521,0.856988,-2.751451,-1.582735,1.672246,0.656438,-0.932473,2.987436,0
3,1.573270,-0.580318,-0.866332,-0.603812,3.125716,0.870321,-0.161992,4.499666,1.038741,-1.092716,...,1.275598,-3.480110,-1.065252,2.153133,1.563539,2.767117,0.215748,0.619645,1.883397,1
4,-0.613071,-0.644204,1.112558,-0.032397,3.490142,-0.011935,1.443521,-4.290282,-1.761308,0.807652,...,-1.803473,0.518579,-0.205029,-4.744566,-1.520015,1.830651,0.870772,-1.894609,0.408332,0


In [33]:
tr.iloc[:,-1] #提取最後一欄資料

0      1
1      0
2      0
3      1
4      0
      ..
995    0
996    1
997    1
998    0
999    0
Name: Target, Length: 1000, dtype: int64

In [34]:
X_train, X_val, y_train, y_val=train_test_split(tr.iloc[:,:-1], tr.iloc[:,-1], stratify=tr["Target"], random_state=45)  

In [35]:
print('X_train:', X_train.shape )
X_train.head()

X_train: (750, 40)


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
869,-0.660608,0.828060,1.406535,0.332745,11.121916,1.432795,-3.901449,0.956720,1.155516,1.096025,...,0.699164,-1.714052,-3.322502,-0.143597,-2.540029,-2.013784,1.008475,-1.305356,1.872620,5.304688
635,-0.118976,1.325943,1.221823,-0.697761,1.082652,-0.206020,0.701217,1.697551,0.582257,-0.298018,...,1.120061,0.757998,1.884964,-0.140445,-3.238736,1.435964,0.474355,1.003659,-0.715812,-0.358117
924,0.267421,0.909542,0.421344,0.970218,-2.843961,0.526093,2.063731,0.424967,-0.556945,2.248597,...,-0.980134,0.368849,-0.614818,-0.157239,2.586724,0.194750,-0.317927,0.307606,-0.513662,0.873475
42,0.925538,0.914358,1.569475,0.024326,8.931533,-0.437528,-0.653657,-0.543715,-0.427445,0.708448,...,-0.650925,-0.851366,-3.999497,0.746706,-2.276114,-1.822178,-0.390981,-0.489374,0.891798,-0.360057
811,1.733568,0.246181,-0.982873,0.758622,-0.365527,-0.994507,1.154942,1.163490,-1.497646,-0.894383,...,-1.279501,2.260088,1.928198,1.832400,1.069979,-1.791462,3.167526,0.538286,-3.876441,-4.313612


In [36]:
print('X_val:', X_val.shape )
X_val.head()

X_val: (250, 40)


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
697,-1.405530,-1.011848,1.374505,0.259405,10.472923,0.982006,-0.657918,-1.501485,0.156644,0.558112,...,1.846014,1.139552,-4.695496,-0.334492,1.716241,-0.338008,-0.113559,-0.528523,-1.756605,-3.862706
607,0.158192,0.919010,0.194450,-1.111271,14.593580,0.025803,0.593168,1.082880,1.137166,0.626944,...,-0.310644,-0.412744,-4.837684,2.015380,-0.533041,0.131701,-4.811711,-0.234398,-1.532204,-3.879218
13,-0.314769,-0.164762,0.722565,-1.256357,0.210247,-0.505173,3.989229,-0.911114,2.236499,0.551156,...,0.949611,-1.918109,-0.801626,-0.827601,2.070438,-1.258529,1.205850,-1.746090,-2.318682,-6.124575
369,0.317380,0.740574,0.379908,-0.273790,-2.471854,-2.022048,-2.377067,-3.285266,0.222391,0.518306,...,-0.625281,0.607746,2.632173,-0.320084,-1.987805,-0.036247,0.215520,-2.073017,-5.925336,3.513265
570,-1.160890,0.891659,0.839922,0.118237,6.109413,0.503881,0.394821,3.612922,1.029285,1.420344,...,2.209174,0.694812,-0.019776,0.071775,-1.498516,1.714792,0.010387,1.509384,-1.196980,-1.538654


In [37]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((750, 40), (250, 40), (750,), (250,))

In [38]:
#RandomForestClassifier

In [39]:
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV

In [40]:
clf = RandomForestClassifier(n_estimators=10, max_depth=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)

acc = metrics.accuracy_score(y_val, y_pred)
print("Accuracy: ", acc)
print("Feature importance: ", clf.feature_importances_)

Accuracy:  0.812
Feature importance:  [0.0082347  0.01595895 0.0049373  0.00876543 0.0364992  0.00930686
 0.05796062 0.00874841 0.01651635 0.01383034 0.00802712 0.01388863
 0.07283455 0.01141284 0.15093093 0.01270083 0.00689232 0.01078375
 0.08257844 0.01494268 0.01046863 0.02283626 0.02489643 0.02274147
 0.01657878 0.00599369 0.01129247 0.01196019 0.02768436 0.02723507
 0.01028148 0.00618075 0.05560148 0.02031613 0.04106352 0.00755595
 0.03158718 0.01108759 0.02417469 0.04471361]


In [41]:
#超參數設定

In [49]:
n_estimators = [50, 100, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

In [50]:
## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="accuracy", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(X_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    5.1s finished


In [51]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.842667 using {'max_depth': 5, 'n_estimators': 300}


In [52]:
grid_result.best_params_

{'max_depth': 5, 'n_estimators': 300}

In [55]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
# 使用最佳參數重新建立模型

clf_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(X_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(X_val)

In [56]:
# 調整參數後約可降至 8.30 的 MSE
print(metrics.accuracy_score(y_val, y_pred))

0.9


In [57]:
pred  = clf_bestparam.predict(ts)
rfc_best_pred = pd.DataFrame(pred)

rfc_best_pred.index += 1

rfc_best_pred.columns = ['Solution']
rfc_best_pred['Id'] = np.arange(1,rfc_best_pred.shape[0]+1)
rfc_best_pred = rfc_best_pred[['Id', 'Solution']]

rfc_best_pred.to_csv('Lindam_2020-08-13.csv',index=False)

In [ ]:
*-------------------------------*

In [43]:
#設定要訓練的超參數組合 
n_estimators = [int(x) for x in np.linspace(10, 2000, 20)] #numpy.linspace(start, stop, num=50
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, 11)] 
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

param_grid = dict(n_estimators=n_estimators, max_features=max_features, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, bootstrap=bootstrap)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = RandomizedSearchCV(clf, param_grid, n_jobs=-1, verbose=1, cv = 10)

# 開始搜尋最佳參數
grid_result = grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   35.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.5min finished


In [44]:
#印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.876000 using {'n_estimators': 1371, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 70, 'bootstrap': False}


In [46]:
# 使用最佳參數重新建立模型
clf_bestparam = RandomForestClassifier(n_estimators     =grid_result.best_params_['n_estimators'],
                                       min_samples_split=grid_result.best_params_['min_samples_split'],
                                       min_samples_leaf =grid_result.best_params_['min_samples_leaf'],
                                       max_features     =grid_result.best_params_['max_features'],
                                       max_depth        =grid_result.best_params_['max_depth'],
                                       bootstrap        =grid_result.best_params_['bootstrap'])
                                       
# 訓練模型
clf_bestparam.fit(X_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(X_val)
acc = metrics.accuracy_score(y_val, y_pred)
print("Accuracy: ", acc)
print("Feature importance: ", clf_bestparam.feature_importances_)

Accuracy:  0.884
Feature importance:  [0.01137388 0.01219934 0.01055671 0.01039471 0.03773203 0.01004664
 0.05382516 0.02090995 0.01147308 0.00906723 0.01040814 0.01107204
 0.11097273 0.01049001 0.12964889 0.01054306 0.00913942 0.00887503
 0.05739108 0.0095384  0.00973668 0.01140496 0.02458896 0.02536983
 0.01010563 0.01188896 0.01069946 0.01002242 0.03661323 0.03643994
 0.01117025 0.01091245 0.04014106 0.0105411  0.03742289 0.00926801
 0.04760716 0.00976551 0.02005326 0.06059073]


In [48]:
pred  = clf_bestparam.predict(ts)
rfc_best_pred = pd.DataFrame(pred)

rfc_best_pred.index += 1

rfc_best_pred.columns = ['Solution']
rfc_best_pred['Id'] = np.arange(1,rfc_best_pred.shape[0]+1)
rfc_best_pred = rfc_best_pred[['Id', 'Solution']]

rfc_best_pred.to_csv('may_2020-08-13.csv',index=False)